In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ""

In [ ]:
from NN import *
from utils import get_tf_session
from data_utils import make_batch, single_np_datapoint_generator

import tensorflow as tf
import matplotlib.pyplot as plt
import time

In [ ]:
train_dataset_files = [
 './data/tfrecords/0_cropped.tfrecord',
 './data/tfrecords/1_cropped.tfrecord',
 './data/tfrecords/2_cropped.tfrecord',
 './data/tfrecords/3_cropped.tfrecord',
 './data/tfrecords/4_cropped.tfrecord',
 './data/tfrecords/5_cropped.tfrecord',
 './data/tfrecords/6_cropped.tfrecord',
 './data/tfrecords/7_cropped.tfrecord',
 './data/tfrecords/8_cropped.tfrecord',
]

validation_dataset_files = ['./data/tfrecords/9_cropped.tfrecord']

In [4]:
X_batch, Y_batch = make_batch(train_dataset_files, 200, raw_values=True)
no_of_outputs = 7
_, age_logits = get_classification_NN(X_batch, 60, [lambda i: 10 + 3*i], 4, no_of_outputs)
loss = tf.losses.sparse_softmax_cross_entropy(tf.cast((Y_batch[:, 1] - 10)/10, tf.int32), age_logits)
train_op = get_training_op(loss)
X = tf.placeholder(dtype=tf.float32, shape=(None, None, None, 3))
age, _ = get_classification_NN(X, 60, [lambda i: 10 + 3*i], 4, no_of_outputs)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [5]:
sess = get_tf_session()
if tf.test.is_gpu_available():
    print("Tensorflow is able to detect gpu...")

Tensorflow is able to detect gpu...


In [6]:
avg_loss = 0
print_int = 200
st = time.time()
# while datetime.datetime.now().time() < datetime.time(hour=8, minute=30):
for i in range(5000):
    _, loss_value= sess.run([train_op, loss])
    avg_loss += loss_value
    if i>0 and i%print_int == 0:
        print("Batch No.", i, "Avg loss:", avg_loss/print_int)
        avg_loss = 0
print("training took total", time.time() - st, "seconds")

Batch No. 200 Avg loss: 1.6737267291545868
Batch No. 400 Avg loss: 1.6345791822671891
Batch No. 600 Avg loss: 1.6235600543022155
Batch No. 800 Avg loss: 1.6120854979753494
Batch No. 1000 Avg loss: 1.5967767149209977
Batch No. 1200 Avg loss: 1.5869263374805451
Batch No. 1400 Avg loss: 1.572634575366974
Batch No. 1600 Avg loss: 1.5580687123537063
Batch No. 1800 Avg loss: 1.544734951853752
Batch No. 2000 Avg loss: 1.530384379029274
Batch No. 2200 Avg loss: 1.5204052072763443
Batch No. 2400 Avg loss: 1.5098592001199722
Batch No. 2600 Avg loss: 1.4971493977308272
Batch No. 2800 Avg loss: 1.481782270669937
Batch No. 3000 Avg loss: 1.469295637011528
Batch No. 3200 Avg loss: 1.4591459798812867
Batch No. 3400 Avg loss: 1.4496593546867371
Batch No. 3600 Avg loss: 1.4403153842687606
Batch No. 3800 Avg loss: 1.4296376419067383
Batch No. 4000 Avg loss: 1.422810165286064
Batch No. 4200 Avg loss: 1.4147529816627502
Batch No. 4400 Avg loss: 1.4079501551389695
Batch No. 4600 Avg loss: 1.396734335422516

In [7]:
gen = single_np_datapoint_generator(validation_dataset_files, raw_values=True)
correct = 0
total = 0
for img, attr in group_iterable_into_list(gen, 20, 2):
    pred = sess.run([age], feed_dict={X: img})[0]
    age_binned = np.floor((np.array(attr)[:, 1] - 10)/10)
    pred_labels = np.argmax(pred, axis=1)
    correct += np.count_nonzero(pred_labels == age_binned)
    total += pred.shape[0]
print("Accuracy", correct/total)

Accuracy 0.3639387890884897
